In [1]:
# Get the data for expeirment
import pandas as pd 
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF
import seaborn as sns
import TsModel
import GprModel
import TsModel_constant
import torch
from sklearn.metrics import r2_score, root_mean_squared_error
import matplotlib.pyplot as plt
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
from keras import layers, models

device = torch.device("cuda:0")
print(f"Keras version is {keras.__version__}")
print(f"Num GPUs Available: {torch.cuda.device_count()}")
plt.rcParams.update({'font.size': 20})

Keras version is 3.1.1
Num GPUs Available: 1


In [18]:
exp27_results = {
    'tsl_train_accuracy' : tsl_train_accuracy,
    'tsl_test_accuracy' :tsl_test_accuracy,
    'tsl_r2s':tsl_r2s,
    'tsl_time': tsl_time,
    'tsc_train_accuracy':tsc_train_accuracy,
    'tsc_test_accuracy':tsc_test_accuracy,
    'tsc_r2s':tsc_r2s,
    'tsc_time':tsc_time,
    'gp_tsl_train_accuracy':gp_tsl_train_accuracy,
    'gp_tsl_test_accuracy':gp_tsl_test_accuracy,
    'gp_tsl_r2s':gp_tsl_r2s,
    'gp_tsl_time':gp_tsl_time,
    'gp_tsc_train_accuracy':gp_tsc_train_accuracy,
    'gp_tsc_test_accuracy':gp_tsc_test_accuracy,
    'gp_tsc_r2s':gp_tsc_r2s,
    'gp_tsc_time':gp_tsc_time,
    'gp_nn_tsl_train_accuracy':gp_nn_tsl_train_accuracy,
    'gp_nn_tsl_test_accuracy':gp_nn_tsl_test_accuracy,
    'gp_nn_tsl_r2s':gp_nn_tsl_r2s,
    'nn_tsl_r2s':nn_tsl_r2s,
    'gp_nn_tsl_time':gp_nn_tsl_time,
    'gp_nn_tsc_train_accuracy':gp_nn_tsc_train_accuracy,
    'gp_nn_tsc_test_accuracy':gp_nn_tsc_test_accuracy,
    'gp_nn_tsc_r2s':gp_nn_tsc_r2s,
    'nn_tsc_r2s':nn_tsc_r2s,
    'gp_nn_tsc_time':gp_nn_tsc_time
}

In [19]:
#%store exp27_results 

Stored 'exp27_results' (dict)


In [2]:
# Train and test the solar data set
x_train = pd.read_csv('Data/FES/solar2_x_train.csv')
x_test = pd.read_csv('Data/FES/solar2_x_test.csv')
y_train = pd.read_csv('Data/FES/solar2_y_train.csv')
y_test = pd.read_csv('Data/FES/solar2_y_test.csv')

In [3]:
number_of_folds = 10

# Source: From the TSmodel
def preprocess_data(train_x, test_x, train_y, test_y):
    feature_scaler = StandardScaler()
    scaled_train_x = feature_scaler.fit_transform(train_x)
    scaled_test_x = feature_scaler.transform(test_x)

    # target_scaler = StandardScaler()  
    # scaled_train_y = target_scaler.fit_transform(train_y)
    # scaled_test_y = target_scaler.transform(test_y)    
    # return scaled_train_x, scaled_test_x, scaled_train_y, scaled_test_y, feature_scaler, target_scaler    

    return scaled_train_x, scaled_test_x, feature_scaler    

def build_neural_network_model():
    model = models.Sequential()
    model.add(layers.Dense(10, activation=keras.activations.tanh))    
    model.add(layers.Dense(10, activation=keras.activations.tanh))    
    model.add(layers.Dense(1, activation=keras.activations.relu))
    return model

In [4]:
x_train, x_test, feature_scaler = preprocess_data(x_train, x_test, y_train, y_test)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [5]:
def do_nn_exp():
    neural_network_model = build_neural_network_model()

    neural_network_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.1),
        loss=keras.losses.MeanSquaredError(),
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    start_time = time.time()
    history = neural_network_model.fit(
        x_train, 
        y_train,
        epochs=200, 
        shuffle=True
    )
    time_used = time.time() - start_time

    print(f'time used: {time_used}')

    return neural_network_model

In [6]:
nn_model = do_nn_exp()

Epoch 1/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 22029274.0000 - root_mean_squared_error: 4633.1914
Epoch 2/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 23854606.0000 - root_mean_squared_error: 4847.4839
Epoch 3/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 21877226.0000 - root_mean_squared_error: 4651.4814
Epoch 4/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 25383368.0000 - root_mean_squared_error: 4946.5376
Epoch 5/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 25476294.0000 - root_mean_squared_error: 5003.2393
Epoch 6/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 27288376.0000 - root_mean_squared_error: 5152.4453
Epoch 7/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 18060844.0000 - root_mean_squared_error: 4223.2319
Epoch 8/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 18415392.0000 - root_mean_squared_error: 4265.3750
Epoch 9/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 15617161.0000 - root_mean_squared_error: 3921.0476
Epoch 10/2

In [7]:
#nn_model.save('solar2_nn.keras')

In [8]:
def get_tsl_model(x_train, y_train):
    tsl_model = TsModel.TsModel(number_of_rules=25, fuzzification_coefficient=1.2)
    tsl_model.fit(x_train, y_train)
    return tsl_model

def get_tsc_model(x_train, y_train):
    tsc_model = TsModel_constant.TsModel_constant(number_of_rules=25, fuzzification_coefficient=1.2)
    tsc_model.fit(x_train, y_train)
    return tsc_model

In [9]:
# TS model
# Linear conclusion

def ts_relu_wrapper(pred):
    return np.maximum(0, pred)

In [10]:
# GPR model
gp_train_accuracy = []
gp_test_accuracy = []
gp_r2s = []
gp_time = []
gpr_model = None

for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)
    indices = np.random.choice(x_train.shape[0], 2000, replace=False)      

    start_time = time.time()
    gpr_model.fit(x_train[indices,:], y_train[indices,:])
    time_used = time.time() - start_time

    train_means = gpr_model.predict(x_train)
    test_means = gpr_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_train_accuracy.append(train_rmse)
    gp_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_r2s.append(gp_r2)
    gp_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_r2s),3)
gp_r2_std = np.round(np.std(gp_r2s),3)
gp_time_mean = np.round(np.mean(gp_time),3)
gp_time_std = np.round(np.std(gp_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-

GP Train RMSE: 1511.79 ± 400.305
GP Test RMSE: 4622.61 ± 104.356
GP R2: -0.095 ± 0.049
GP Time: 13.485 ± 6.02


In [11]:
# GPR model
gp_tsl_train_accuracy = []
gp_tsl_test_accuracy = []
gp_tsl_r2s = []
gp_tsl_time = []
gpr_tsl_model = None

# From ts linear
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_tsl_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)
    tsl_model = get_tsl_model(x_train, y_train)

    start_time = time.time()
    gpr_tsl_model.fit(tsl_model.cen, tsl_model.predict(tsl_model.cen + 1e-8))
    time_used = time.time() - start_time

    train_means = gpr_tsl_model.predict(x_train)
    test_means = gpr_tsl_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_tsl_train_accuracy.append(train_rmse)
    gp_tsl_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_tsl_r2s.append(gp_r2)
    gp_tsl_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_tsl_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_tsl_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_tsl_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_tsl_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_tsl_r2s),3)
gp_r2_std = np.round(np.std(gp_tsl_r2s),3)
gp_time_mean = np.round(np.mean(gp_tsl_time),3)
gp_time_std = np.round(np.std(gp_tsl_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


FCM training RMSE: 3635.218885720649


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 3657.5230808049027


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 3654.872497167509


...

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 3661.2449588560144


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 3671.6209057502324
GP Train RMSE: 3796.438 ± 4.23
GP Test RMSE: 4042.684 ± 4.172
GP R2: 0.163 ± 0.002
GP Time: 0.028 ± 0.006


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [13]:
# GPR model
gp_tsc_train_accuracy = []
gp_tsc_test_accuracy = []
gp_tsc_r2s = []
gp_tsc_time = []
gpr_tsc_model = None

# From ts constant
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_tsc_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)
    tsc_model = get_tsc_model(x_train, y_train)
    start_time = time.time()
    gpr_tsc_model.fit(tsc_model.cen, tsc_model.predict(tsc_model.cen + 1e-8))
    time_used = time.time() - start_time

    train_means = gpr_tsc_model.predict(x_train)
    test_means = gpr_tsc_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_tsc_train_accuracy.append(train_rmse)
    gp_tsc_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_tsc_r2s.append(gp_r2)
    gp_tsc_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_tsc_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_tsc_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_tsc_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_tsc_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_tsc_r2s),3)
gp_r2_std = np.round(np.std(gp_tsc_r2s),3)
gp_time_mean = np.round(np.mean(gp_tsc_time),3)
gp_time_std = np.round(np.std(gp_tsc_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


(2281, 150)
[[9.96041354e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.42248410e-11 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.86430908e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.36426979e-11 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.11833275e-10 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.55965769e-12 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2281, 25)
[[9.96041354e-01 1.38062393e-09 9.87134663e-08 ... 3.83958846e-07
  1.64562467e-06 2.29733003e-07]
 [1.42248410e-11 9.99952995e-01 5.85408449e-09 ... 2.54778229e-11
  2.63274270e-11 2.69862152e-10]
 [5.86430908e-04 6.77137582e-08 4.07420799e-05 ... 2.80422040e-06
  1.99739927e-06 3.94027960e-04]
 ...
 [3.36426979e-11 1.19716438e-08 1.13753106e-06 ... 1.89617630e-11
  2.36556

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(2281, 150)
[[5.26476085e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.27916845e-10 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.18701673e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.44918202e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.69176451e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.56575775e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2281, 25)
[[5.26476085e-09 9.35478976e-09 9.53820377e-05 ... 1.22592315e-09
  9.99578835e-01 3.59915953e-07]
 [2.27916845e-10 1.80104811e-07 3.63066089e-11 ... 9.99946211e-01
  4.73288857e-12 2.02804392e-10]
 [1.18701673e-06 5.20350623e-07 6.73716691e-05 ... 5.49054079e-08
  2.69377371e-04 8.20031316e-04]
 ...
 [1.44918202e-07 2.10368665e-09 1.10623525e-10 ... 1.25935031e-08
  1.24677

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(2281, 150)
[[2.11256482e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.41561424e-10 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.84874052e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.61454696e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.60219764e-08 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.25598242e-12 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2281, 25)
[[2.11256482e-09 1.26539210e-09 1.31035165e-05 ... 1.26211070e-08
  2.92769868e-05 6.41059324e-09]
 [1.41561424e-10 4.06121722e-09 4.47263135e-10 ... 1.44015616e-09
  4.01644420e-12 5.91854046e-09]
 [3.84874052e-06 8.43871583e-07 1.85487236e-03 ... 1.85206828e-06
  8.95689949e-06 4.60774628e-06]
 ...
 [2.61454696e-09 1.76213743e-10 6.19118075e-10 ... 1.54604390e-10
  2.30638

...

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(2281, 150)
[[1.47046850e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.52070210e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.82488963e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [9.99923791e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.99597536e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.26492435e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2281, 25)
[[1.47046850e-09 3.31104612e-09 4.75776648e-05 ... 9.88718575e-10
  5.61943196e-05 2.92568894e-07]
 [1.52070210e-09 4.72341873e-11 1.41695295e-12 ... 9.99943397e-01
  5.49194281e-11 2.22745611e-10]
 [6.82488963e-07 2.89565429e-06 7.80528215e-06 ... 2.25408917e-07
  4.23071945e-04 3.37582157e-03]
 ...
 [9.99923791e-01 2.70747085e-09 2.34207176e-12 ... 1.20608747e-08
  1.67060

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(2281, 150)
[[9.73775100e-10 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.99947431e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.96481198e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.21898976e-08 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.73161085e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.02539096e-10 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(2281, 25)
[[9.73775100e-10 2.76729153e-06 2.30907884e-08 ... 4.81277281e-05
  2.45312668e-09 4.83264798e-08]
 [9.99947431e-01 4.26805086e-10 4.63424517e-10 ... 1.29965444e-12
  7.44266962e-08 1.12428037e-10]
 [1.96481198e-07 7.11695545e-04 1.51185701e-06 ... 6.87023293e-06
  8.52226838e-07 5.76280010e-06]
 ...
 [1.21898976e-08 2.91205279e-09 1.57933236e-10 ... 2.34086141e-12
  1.25517

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [14]:
# GPR model
gp_nn_tsl_train_accuracy = []
gp_nn_tsl_test_accuracy = []
gp_nn_tsl_r2s = []
nn_tsl_r2s = []
gp_nn_tsl_time = []
gpr_nn_tsl_model = None

# From NN and ts linear
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_nn_tsl_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)
    tsl_model = get_tsl_model(x_train, y_train)

    start_time = time.time()
    inputs = tsl_model.cen
    ts_predictions = tsl_model.predict(tsl_model.cen + 1e-8)
    nn_predictions = nn_model.predict(inputs)
    gpr_nn_tsl_model.fit(inputs, nn_predictions)
    time_used = time.time() - start_time

    train_means = gpr_nn_tsl_model.predict(x_train)
    test_means = gpr_nn_tsl_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_nn_tsl_train_accuracy.append(train_rmse)
    gp_nn_tsl_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_nn_tsl_r2s.append(gp_r2)
    gp_nn_tsl_time.append(time_used)
    nn_tsl_r2s.append(r2_score(ts_predictions, nn_predictions))

gp_train_accuracy_mean = np.round(np.mean(gp_nn_tsl_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_nn_tsl_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_nn_tsl_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_nn_tsl_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_nn_tsl_r2s),3)
gp_r2_std = np.round(np.std(gp_nn_tsl_r2s),3)
gp_time_mean = np.round(np.mean(gp_nn_tsl_time),3)
gp_time_std = np.round(np.std(gp_nn_tsl_time),3)
nn_ts_r2_mean = np.round(np.mean(nn_tsl_r2s),3)
nn_ts_r2_std = np.round(np.std(nn_tsl_r2s),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")
print(f"TS NN R2: {nn_ts_r2_mean} ± {nn_ts_r2_std}")


FCM training RMSE: 3661.599026404337
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 3672.8288663771586
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 3655.963172750529
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step


...

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 3638.720078796616
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 3655.247876925388
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
GP Train RMSE: 3842.186 ± 5.223
GP Test RMSE: 4056.844 ± 4.894
GP R2: 0.157 ± 0.002
GP Time: 0.032 ± 0.017
TS NN R2: 0.892 ± 0.016


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
